In [1]:
import scipy as sp
import pandas as pd
import numpy
import warnings
from matplotlib import pyplot as plt
import surprise

warnings.filterwarnings('ignore')

data = pd.read_csv('./Chicken_review.csv',encoding='CP949')

In [2]:
#data 구조 출력 Colum - Restaurant - UserID - Menu - Review - Total
print(data.tail())

#data 구조 to pandas DataFrame
df = data[['UserID','Menu','Total']]

      Restaurant UserID                                               Menu  \
29945      후아바베큐  sr**님             허브 후라이드 /1(허브 후라이드 양념 선택(양념 반＋후라이드 반))   
29946      후아바베큐  za**님               순살 숯불양념바베큐/1(양념 바베큐 맛 선택(양념 바베큐 달콤))   
29947      후아바베큐  fa**님  순살 허브바베큐 반 ＋순살 숯불양념바베큐 반/1(양념 바베큐 맛 선택(양념 바베큐 ...   
29948      후아바베큐  ge**님  순살 숯불양념바베큐 반 ＋순살 간장골드바베큐 반/1(양념 바베큐 맛 선택(양념 바베...   
29949      후아바베큐  se**님      순살 숯불양념바베큐/1(양념 바베큐 맛 선택(양념 바베큐 매콤)),감자튀김 R/1   

                                                  Review  Total  Taste  \
29945                           후라이드는 옛날통닭같고 양념은 매콤하니 맛남      5    NaN   
29946                                              맛있어요!      5    NaN   
29947  반반시킨건데..친구랑 먹기엔 좀 부족함.. 바베큐치킨은 다 이런건가여... 10조각...      3    NaN   
29948  너무 맛있어요 단 배달이 좀 늦는것(거리가 있어서 당연하다 생각합니다) 양이 너무 ...      5    NaN   
29949                                      양도 적당하고 맛도 있고      5    NaN   

       Quantity  Delivery  
29945       NaN       NaN  
29946       NaN       NaN  
29

In [3]:
#문자열 '/' 기준 앞의 요소들만 남기고 삭제. ※데이터 형태 :  메인메뉴 / 사이드 
for index, value in enumerate(df['Menu']):
    df['Menu'][index] = df['Menu'][index].split('/')[0]

In [4]:
df.tail()

,UserID,Menu,Total
29945,sr**님,허브 후라이드,5
29946,za**님,순살 숯불양념바베큐,5
29947,fa**님,순살 허브바베큐 반 ＋순살 숯불양념바베큐 반,3
29948,ge**님,순살 숯불양념바베큐 반 ＋순살 간장골드바베큐 반,5
29949,se**님,순살 숯불양념바베큐,5


In [5]:
df.to_excel('review_frameone.xlsx')

In [6]:
df_table = df.set_index(['UserID'],['Menu']).unstack()
df_table.shape

(59900,)

In [7]:
#data to Dictionary 함수
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1 : return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.ix[:,1:]) for k,g in grouped}
    return d

In [8]:
#df_to_dict 변수에 dataframe을 사전 형식으로 저장
df_to_dict = recur_dictify(df)

In [10]:
df_to_dict.to_excel('review_frameone.xlsx')

AttributeError: 'dict' object has no attribute 'to_excel'

In [11]:
a = pd.DataFrame(df_to_dict.items(),columns=['UserID','Menu','Total'])
print(a.head())

ValueError: 3 columns passed, passed data had 2 columns

In [12]:
df_to_dict

{'-3**님': {'반반치킨': 5},
 '-_**님': {'투마리세트': 5, '후라이드': array([5, 5, 5, 5], dtype=int64), '후라이드치킨': 5},
 '-m**님': {'크리미언반＋핫블링반': 5},
 '-w**님': {'맛초킹（한마리）': 5},
 '00**님': {'더화이트치킨 슈프림': 5,
  '마늘 간장치킨': 4,
  '반반 치킨': 5,
  '순살 슈프림양념치킨': array([5, 5], dtype=int64),
  '순살 후라이드': 5,
  '순살치킨': 4,
  '알로하 오븐소금구이': 5,
  '양념치킨': 1,
  '윙봉반반（슈프림반＋양념반）': 5,
  '크리미언치킨': 4,
  '한마리세트（한마리 택1＋똥집＋콜라500mL）': 2},
 '01**님': {'간장소스치킨＋콜라': 5,
  '갈릭플러스반＋핫양념반': 3,
  '갈릭플러스치킨': 5,
  '국물떡볶이 치킨세트': 5,
  '굽네 고추 바사삭': 2,
  '다리 윙봉 슈프림 ': 5,
  '닭다리': 5,
  '두마리 치킨': 5,
  '마왕치킨': 5,
  '매운 양념 치킨': 5,
  '목우촌 닭다리': 4,
  '반반 치킨 （후라이드＋ 양념）': 5,
  '반반치킨': array([5, 5, 5, 5, 5, 5, 2, 4, 4, 5], dtype=int64),
  '반반치킨 순살': 5,
  '반반치킨（양념반＋와락간장반）': 5,
  '반반치킨（후라이드 반＋양념 반）': 5,
  '반반치킨（후라이드반＋양념반）': 5,
  '뼈한마리 ＋ 뼈한마리': 5,
  '순살 두마리세트（순살치즈슈프림양념치킨＋순살와락간장치킨＋콜라1.25l）': 4,
  '순살 두마리세트（순살후라이드＋순살후라이드＋콜라1.25l）': 5,
  '순살 두마리세트（후라이드＋핫슈프림양념치킨＋콜라 1.25l）': 5,
  '순살 반반치킨': 4,
  '순살 허브바베큐 반 ＋순살 숯불양념바베큐 반': 5,
  '순살3종세트（후라이드＋양념＋깐풍）': 5,
  '순살두마리 세트': 

In [13]:
a = pd.DataFrame(list(df_to_dict.items()),columns=['UserID','Menu','Total'])
print(a.head())

ValueError: 3 columns passed, passed data had 2 columns

In [14]:
a = pd.DataFrame(df_to_dict.items(),columns=['UserID','Menu'])
print(a.head())

  UserID                                               Menu
0  -3**님                                        {'반반치킨': 5}
1  -_**님    {'투마리세트': 5, '후라이드': [5, 5, 5, 5], '후라이드치킨': 5}
2  -m**님                                  {'크리미언반＋핫블링반': 5}
3  -w**님                                    {'맛초킹（한마리）': 5}
4  00**님  {'더화이트치킨 슈프림': 5, '마늘 간장치킨': 4, '반반 치킨': 5, '순...


In [15]:
a.to_excel('review_dictone.xlsx')

In [16]:
#UserId를 담을 리스트 (중복X)
name_list = []
#치킨 목록을 담을 set (중복O)
chk_set = set()

for user_key in df_to_dict:
    name_list.append(user_key)
    
    for chk_key in df_to_dict[user_key]:
        chk_set.add(chk_key)

In [17]:
chk_list = list(chk_set)

In [18]:
#사전화 된 각각의 Colum들을 indexing 하여 저장할 Dictionary
rating_dic = {
    'UserID' : [],
    'Menu' : [],
    'Total' : []
}

#사용자 수 만큼 반복
for name_key in df_to_dict :
    #사용자가 구매한 치킨의 수 만큼 반복
    for chk_key in df_to_dict[name_key]:
        
        #사용자 인덱스 번호 추출
        a1 = name_list.index(name_key)
        #치킨 메뉴 인덱스 번호 추출
        a2 = chk_list.index(chk_key)
        #사용자-치킨메뉴 의 평점 추출 
        a3 = df_to_dict[name_key][chk_key]
        
        #Dictionary에 저장
        rating_dic['UserID'].append(a1)
        rating_dic['Menu'].append(a2)
        #만약 사용자 ID 중복으로 인해 한 메뉴에 여러 평점이 들어갔으면, 첫번째 평점만 저장
        if type(a3) is not numpy.ndarray:
            rating_dic['Total'].append(a3)
        else:
            rating_dic['Total'].append(a3[0])

In [19]:
df = pd.DataFrame(rating_dic)

In [20]:
df.tail()

,UserID,Menu,Total
18059,894,799,5
18060,894,314,5
18061,894,625,5
18062,894,770,5
18063,894,337,5


In [21]:
df.to_excel('rating_dictone.xlsx')

In [22]:
#reader 객체로 Data읽기. rating scale은 평점의 범위 `1~5`
reader = surprise.Reader(rating_scale=(1,5))

#Surprise 에서 사용할 데이터 셋 userid=id, menu=치킨메뉴, Total=평점
col_list = ['UserID','Menu','Total']
data = surprise.Dataset.load_from_df(df[col_list], reader)

#Trainset에 입력한 Data로 학습
trainset = data.build_full_trainset()
option = {'name':'pearson'}
algo = surprise.KNNBasic(sim_options=option) #KNN 유사도 알고리즘 사용

algo.fit(trainset)

# 치킨 추천 닉네임은 csv파일에 있는 id 아무거나 쓰면됨 양식 'hw**님' ※'님' 까지 붙여야함
who = input('input nickname : ')
print('\n')

index = name_list.index(who)
print('user_index:',index)
print('\n')

result = algo.get_neighbors(index, k=5) #Knn 모델의 K값 '5'로 지정 유사사용자 5명으로부터 화장품 추천
print('치킨 취향이 비슷한 사용자 : ',result)
print('\n')

print('추천 치킨:','\n')

for r1 in result :
    max_rating=data.df[data.df["UserID"]==r1]["Total"].max()
    chk_id=data.df[(data.df["Total"]==max_rating)&(data.df["UserID"]==r1)]["Menu"].values
    print(chk_id)
    
    for chk_item in chk_id:
        print(chk_list[chk_item])

Computing the pearson similarity matrix...
Done computing similarity matrix.
input nickname : 01**님


user_index: 5


치킨 취향이 비슷한 사용자 :  [28, 39, 94, 236, 302]


추천 치킨: 

[ 846  645  117  909  415   72  601  904  665  302 1014]
교촌오리지날
깐풍치킨
반반오리지날
반반치킨
소이갈릭치킨
순살양념구이치킨
양념반＋후라이드반
양념치킨
커리퀸다리¸
크리미언반＋소이갈릭반
허니오리지날
[ 899  344  148  117  909 1086  485  229  235  126  713  817]
간장치킨 반마리
더화이트치킨 슈프림
맛초킹＋치즈볼
반반오리지날
반반치킨
순살두마리 세트
스노윙반＋양념반
스노윙（치즈）반＋쇼킹핫반
취향!저격! 후반＋양반
크리미언반＋핫블링반
핫슈프림 양념치킨
후라이드치킨 ＋ 매운간장소스치킨＋콜라
[909]
반반치킨
[651 647 276]
매콤치즈스노윙반＋후라이드반
뿌링클
순살매콤치즈스노윙반＋순살후라이드반
[ 157  651 1088  647  870  188  493 1086  629  453 1093  382  780  679
  220  625  612]
국물떡볶이 치킨세트
매콤치즈스노윙반＋후라이드반
뼈한마리 ＋ 뼈한마리
뿌링클
순살 두마리 치킨
순살3종세트（후라이드＋양념＋깐풍）
순살강정치킨
순살두마리 세트
순살매콤치즈스노윙반＋순살양념반
순살반반치킨（양념반＋와락간장반）
순살치킨
순살치킨 ＋ 순살치킨
양동시장통닭
양동양념후라이드
자메이카 통다리구이
투마리세트
후라이드 반 ＋ 양념 반 치킨
